## Loading Libraries 

In [185]:
#Importing Libraries

from PIL import Image
# import simplejson as json 
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib
import requests
import time
from datetime import date
import http.client, urllib.request 
import urllib.parse, urllib.error 
import base64, sys 
import pandas as pd

## Sign in Function

In [186]:
def signin(file='config.txt'):

    driver.get('https://www.linkedin.com/login')

    #Login credentials for Linkedin
    file=open(file)
    lines=file.readlines()
    username=lines[0]
    password=lines[1]

    # locate username and password form login page by HTML id
    enter_username = driver.find_element_by_id('username')
    enter_password=driver.find_element_by_id('password')

    # send_keys() to simulate key strokes
    enter_username.send_keys(username)
    enter_password.send_keys(password)

    #Submit Credentials
    enter_password.submit()


## Scroll Function

In [187]:
def scroll():
    SCROLL_PAUSE_TIME = 2
    
    #Scroll to see more and click if visible
    driver.execute_script("window.scrollTo(0,560);")
    time.sleep(SCROLL_PAUSE_TIME)
    
    try:
        driver.find_element_by_id('line-clamp-show-more-button').click()
    except:
        pass
    
    #Scroll back to top 
    
    driver.execute_script("window.scrollTo(0,0);")
    time.sleep(SCROLL_PAUSE_TIME)

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, "html.parser")
    return soup


## Funcitions for Scraping 

In [188]:
def Scrape_name():
    name_all= soup.find('div',{'class':'flex-1 mr5'})#
    name=name_all.find('li',{'class':'inline t-24 t-black t-normal break-words'}).get_text().strip()
    return name

In [189]:
def Scrape_image():
    imgtag=soup.select('img[title]')
    urllib.request.urlretrieve(imgtag[0]['src'],'scraped_image.jpg')
    return imgtag[0]['src']

In [190]:
def Scrape_about():
    try:
        about_all= soup.find('span',{'class':'lt-line-clamp__raw-line'}).get_text().strip()
        return about_all
    except:
        return 'None'


In [191]:
def Scrape_header():
    name_all= soup.find('div',{'class':'flex-1 mr5'})
    header=name_all.find_all('h2')[0].get_text().strip()
    return header

In [192]:
def Scrape_location():
    name_all= soup.find('div',{'class':'flex-1 mr5'})
    location=name_all.find('li',{'class':'t-16 t-black t-normal inline-block'}).get_text().strip()
    return location

In [193]:
def Scrape_connections():
    name_all= soup.find('div',{'class':'flex-1 mr5'})
    try:
        connections=name_all.find('span',{'class':'t-16 t-bold link-without-visited-state'}).get_text().strip()
        return connections
    except:
        pass
    connections=name_all.find('span',{'class':'t-16 t-black t-normal'}).get_text().strip()
    return connections


In [194]:
def Scrape_joining():
    joining='Present'
    
    try:
        company_all= soup.find('div',{'class':'pv-entity__company-summary-info'})
        experience=company_all.find_all('h4')[0].get_text().strip().split('\n')[1]
        return joining
    except:
        pass
    
    try:
        company_all= soup.find('div',{'class':'pv-entity__summary-info pv-entity__summary-info--background-section'})
        joining=company_all.find_all('span')[2].get_text().strip()
    except:
        pass
    
    return joining

In [195]:
def Scrape_experience():
    experience='Unknown'
    
    try:
        company_all= soup.find('div',{'class':'pv-entity__company-summary-info'})
        experience=company_all.find_all('h4')[0].get_text().strip().split('\n')[1]
        return experience
    except:
        pass
    
    try:
        company_all= soup.find('div',{'class':'pv-entity__summary-info pv-entity__summary-info--background-section'})
        experience=company_all.find_all('span')[4].get_text().strip()    
    except:
        pass
    
    return experience


In [196]:
experience

['Unknown']

In [197]:
def Scrape_company():
    company='Unknown'
    
    try:
        company_all= soup.find('div',{'class':'pv-entity__company-summary-info'})
        company=company_all.find_all('h3')[0].get_text().strip().split('\n')[1]
        return company
    except:
        pass
    
    try:
        company_all= soup.find('div',{'class':'pv-entity__summary-info pv-entity__summary-info--background-section'})
        company=company_all.find_all('p')[1].get_text().strip().split('\n')[0]
    except:
        pass
    
    return company

In [198]:
def Scrape_title():
    title='Unknown'
    
    try:
        company_all= soup.find('div',{'class':'pv-entity__summary-info-v2 pv-entity__summary-info--background-section pv-entity__summary-info-margin-top mb2'})
        title=company_all.find_all('h3')[0].get_text().strip().split('\n')[1]
        return title
    except:
        pass
        
    try:
        company_all= soup.find('div',{'class':'pv-entity__summary-info pv-entity__summary-info--background-section'})
        title=company_all.find_all('h3')[0].get_text().strip()
    except:
        pass
    
    return title

In [199]:
def Scrape_college():
    college_all= soup.find('div',{'class':'pv-entity__degree-info'})
    college=college_all.find_all('h3')[0].get_text().strip()
    return college

In [200]:
def Scrape_degree():
    college_all= soup.find('div',{'class':'pv-entity__degree-info'})
    degree=college_all.find_all('span')[1].get_text().strip()
    return degree

In [201]:
def Scrape_degree_year():
    college_all= soup.find_all('time')
    year1=college_all[0].get_text().strip()
    year2=college_all[1].get_text().strip()
    year=year1 +'-'+ year2
    return year

## Login to Linkedin and enter profiles to scrape

In [205]:
df1=pd.read_csv('Link.csv')

dates=[]
name=[]
image=[]
about=[]
header=[]
location=[]
connections=[]
joining=[]
experience=[]
company=[]
title=[]
college=[]
degree=[]
degree_year=[]

In [206]:
driver = webdriver.Chrome()
driver.maximize_window()
signin()
for link in df1.URL.values:
    Linkedin_url = link
    driver.get(Linkedin_url)
    soup=scroll()
    dates.append(pd.datetime.today().strftime('%d %b, %Y'))
    
    #Scrapping data and appending it to lists
    name.append(Scrape_name())
    image.append(Scrape_image())
    about.append(Scrape_about())
    header.append(Scrape_header())
    location.append(Scrape_location())
    connections.append(Scrape_connections())
    joining.append(Scrape_joining())
    experience.append(Scrape_experience())
    company.append(Scrape_company())
    title.append(Scrape_title())
    college.append(Scrape_college())
    degree.append(Scrape_degree())
    degree_year.append(Scrape_degree_year())

<ipython-input-206-bf7595d1e3bf>:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  dates.append(pd.datetime.today().strftime('%d %b, %Y'))


## Write to csv

In [207]:

linkedin = {'date':dates,'name':name,'image':image,'about':about,'header':header,'location':location,'connections':connections,\
            'joining':joining,'experience':experience,'company':company,'title':title,'college':college,'degree':degree,\
            'degree_year':degree_year}  
    
linkedin_df = pd.DataFrame(linkedin) 
linkedin_df.to_csv('Linkedin_profiles.csv')
linkedin_df

,date,name,image,about,header,location,connections,joining,experience,company,title,college,degree,degree_year
0,"19 Aug, 2020",Shweta Sharma,https://media-exp1.licdn.com/dms/image/C5603AQ...,None,Data Science Consultant at Arcadis,"Clayton, Victoria, Australia",500+ connections,Nov 2019 – Present,10 mos,Arcadis,Data Science Consultant,Monash University,Master of Data Science,2018-2020
1,"19 Aug, 2020",Rohan Singh,https://media-exp1.licdn.com/dms/image/C5635AQ...,Recently completed Masters of Data Science fro...,Data Scientist Intern at AI Australia,"Malvern East, Victoria, Australia",385 connections,Aug 2020 – Present,1 mo,AI Australia,Data Scientist Intern,Monash University,Master of Data Science,2018-2020
2,"19 Aug, 2020",Cody Middlebrook,https://media-exp1.licdn.com/dms/image/C5103AQ...,"Most quickly identified by my Filmmaking, doub...",The AI Guy | Reimagining Artificial Intelligen...,"Sydney, Australia",500+ connections,Present,1 yr 7 mos,AI Films,Interim CEO,London Business School,Diploma in Computer Programming,1997-2000
